<a href="https://colab.research.google.com/github/joshlevin91/Benefit-Corp-Web-Scraper/blob/master/BenefitCorpWebScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Get list of benefit corporation names and websites**



In [ ]:
from bs4 import BeautifulSoup
import requests

# Get names and websites of benefit corporations
def getNamesAndWebsites(url):
  source = requests.get(url).text
  soup = BeautifulSoup(source, 'lxml')
  res = soup.find("div", {"class":"view-content"})

  corporations = []
  corps = res.find_all(class_=None)
  for corp in corps:
    name = corp.find("div", {"class":"bcorps--name"})
    a = corp.find('a', href=True)
    if name and a:
      corporation = [name.text, a['href']]
      corporations.append(corporation)

  return corporations

# Go through all pages to get names and websites of benefit corporations
def goThroughPages(root_url):
    corporations = [] 
    while True:
        corporations.extend(getNamesAndWebsites(root_url))

        # Go to next page
        r = requests.get(root_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        next_page = soup.find("li", {"class": "pager-next"})
        if next_page:
            next_url = next_page.find("a", href=True)
            if next_url:
                root_url = 'https://benefitcorp.net' + next_url['href']
            else:
                break
        else:
            break

    return corporations

# Get list of corporation names and websites
corporations = goThroughPages("https://benefitcorp.net/businesses/find-a-benefit-corp?sort_by=title&sort_order=ASC")

**Write list to a Google worksheet**

In [ ]:
import time

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Create and open worksheet
gc.create('Benefit Corps')
worksheet = gc.open('Benefit Corps').sheet1

# Add corporation data to worksheet
worksheet.insert_row(['Name', 'Website'])
worksheet.format('A1:B1', {'textFormat': {'bold': True}})
for c in corporations:
  time.sleep(0.1) #  Google Sheets API has a limit on the request rate
  worksheet.append_row(c)

**Go to https://sheets.google.com to see your new spreadsheet.**